In [93]:
# Question 3

using Statistics
using CSV
using DataFrames
using LinearAlgebra
using Dates


In [94]:
# Question 4
filepath = "C:/Users/Matt/Downloads/4.5_month_earthquakes.csv"
df = CSV.read(filepath, DataFrame, dateformat="dd-mm-yyyy")
df2 = CSV.read(filepath, DataFrame, dateformat="dd-mm-yyyy", limit = 10)

Row,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
,String31,Float64,Float64,Float64,Float64,String3,Int64,Int64,Float64,Float64,String3,String15,String31,String,String15,Float64,Float64,Float64,Int64,String15,String3,String3
1,2023-01-27T20:25:27.038Z,10.7145,125.448,69.238,5.4,mww,190,26,3.624,0.84,us,us6000jjtj,2023-01-27T22:28:46.168Z,"32 km ENE of Hingatungan, Philippines",earthquake,7.83,5.358,0.066,22,reviewed,us,us
2,2023-01-27T18:18:00.454Z,-2.8446,129.618,20.664,4.7,mb,92,62,2.628,0.6,us,us6000jjsz,2023-01-27T20:31:54.040Z,"95 km NE of Amahai, Indonesia",earthquake,4.25,4.631,0.066,78,reviewed,us,us
3,2023-01-27T16:37:39.419Z,-23.7222,-179.46,525.778,4.6,mb,33,122,4.686,0.53,us,us6000jjsa,2023-01-27T17:00:08.040Z,south of the Fiji Islands,earthquake,16.53,18.134,0.1,30,reviewed,us,us
4,2023-01-27T16:33:28.276Z,-23.3301,-179.886,528.172,4.5,mb,28,115,4.841,0.64,us,us6000jjs9,2023-01-27T17:14:45.040Z,south of the Fiji Islands,earthquake,14.76,10.621,0.118,21,reviewed,us,us
5,2023-01-27T11:06:54.630Z,18.7795,-64.3253,33.0,4.6,ml,21,217,0.7621,0.46,pr,pr2023027001,2023-01-27T18:56:35.897Z,"70 km NE of Cruz Bay, U.S. Virgin Islands",earthquake,2.76,9.97,0.08,13,reviewed,pr,pr
6,2023-01-27T10:51:18.438Z,-16.5691,-69.7504,195.543,4.5,mb,69,105,1.047,0.82,us,us6000jjn6,2023-01-27T14:03:57.040Z,"19 km N of Mazo Cruz, Peru",earthquake,7.09,4.387,0.078,48,reviewed,us,us
7,2023-01-27T05:29:21.314Z,-49.5504,117.454,10.0,5.0,mb,32,145,16.609,0.56,us,us6000jjm1,2023-01-27T05:47:50.040Z,western Indian-Antarctic Ridge,earthquake,15.42,1.926,0.107,28,reviewed,us,us
8,2023-01-27T04:34:38.938Z,5.0199,-76.2029,103.42,4.5,mb,40,95,1.343,0.7,us,us6000jjlq,2023-01-27T12:24:35.798Z,"14 km NNE of San José del Palmar, Colombia",earthquake,7.06,8.868,0.085,40,reviewed,us,us
9,2023-01-27T03:21:35.761Z,13.7912,120.548,110.676,4.5,mb,34,111,8.312,0.5,us,us6000jjlk,2023-01-27T04:14:12.040Z,"10 km WSW of Calatagan, Philippines",earthquake,8.0,8.337,0.08,46,reviewed,us,us


In [95]:
# Question 5
magnitudes = df.mag
println("Min magnitude: [$(minimum(magnitudes))]")
println("Max magnitude: [$(maximum(magnitudes))]")
println("Average magnitude: [$(sum(magnitudes) / length(magnitudes))]")

Min magnitude: [4.5]
Max magnitude: [7.6]
Average magnitude: [4.821545454545453]


In [96]:
# Question 6
for x in 1:length(df.time)
    df.time[x] = df.time[x][1:19]
end
for x in 1:10
    println("$(df.time[x])")
end

2023-01-27T20:25:27
2023-01-27T18:18:00
2023-01-27T16:37:39
2023-01-27T16:33:28
2023-01-27T11:06:54
2023-01-27T10:51:18
2023-01-27T05:29:21
2023-01-27T04:34:38
2023-01-27T03:21:35
2023-01-27T00:10:43


In [97]:
# Question 7

function inTimeRange(N, i)
    present = Dates.now()
    # println((Dates.value(present) - Dates.value(DateTime(df.time[x], DateFormat("yyyy-mm-ddTHH:MM:SS")))))
    dayPassed = ((Dates.value(present) - Dates.value(DateTime(df.time[i], DateFormat("yyyy-mm-ddTHH:MM:SS.sssZ"))))  / (1000 * 3600 * 24))
    # println("the day passed for index $i is $dayPassed")
    return dayPassed <= N
end

function avgMagnitudeNDays(N)  
    sumOfMag = 0
    i = 1
    count = 0
    for x in eachrow(df)
        # println("at index $i")
        if (inTimeRange(N, i))
            sumOfMag += df.mag[i]
            count += 1
        end
        i += 1
    end
    return (sumOfMag / count)
end

# avgMagnitudeNDays(20)

for x in [10, 20, 30, 40 ,50] 
    res = avgMagnitudeNDays(x)
    println("This is in $x days")
    println(res)
end


This is in 10 days
NaN
This is in 20 days
4.849532710280376
This is in 30 days
4.832542372881357
This is in 40 days
4.8270242914979695
This is in 50 days
4.821545454545446


In [98]:
# Question 8
function convertNToDegree(n) #n is in km
    return (n /= 100)
end

function avgMagnitudeWithinNKmOfLatLong(N, latitude, longitude)
    n = convertNToDegree(N)
    sumOfMag = 0
    count = 0
    for x in eachrow(df) 
        distance = sqrt((x.latitude - latitude)^2 + (x.longitude - longitude)^2)
        if (distance < n)
            sumOfMag +=  x.mag
            count += 1
        end
    end
    return sumOfMag / count
end

println(avgMagnitudeWithinNKmOfLatLong(2000, 29, 14))
println(avgMagnitudeWithinNKmOfLatLong(65000, 127, 123))
println(avgMagnitudeWithinNKmOfLatLong(3000, 10, 142))
println(avgMagnitudeWithinNKmOfLatLong(3000, -47, 189))

4.8687499999999995
4.821545454545446
4.79635416666667
4.928571428571429


In [102]:
# Problem 9

# function filtering out the regions that are in the radius   
function isInRadius(row, N, latitude, longitude)
    n = convertNToDegree(N)
    distance = sqrt((row.latitude - latitude)^2 + (row.longitude - longitude)^2)
    if (distance < n) # the place is in the radius
        return true
    end
    return false
end

# function to find the highest magnitude region among the ones in the radius
function highestMagnitude(df)
    highestMagnitude = 0
    result = ""
    for x in eachrow(df) 
        if x.mag > highestMagnitude
            highestMagnitude = x.mag
            result = x.place
        end
    end
    return result
end
function mostActiveRegion(N, latitude , longitude)
    # searching for the most active region
    df_modified = filter(row -> isInRadius(row, N, latitude, longitude), df)
    #finding the max maginitude region inside the radius and return its name
    df_modified
    #df_mod2 = combine(groupby(df_modified, :place), :mag => sum => :mag)
    
    #mostActiveRegion = highestMagnitude(df_mod2)
    #return mostActiveRegion
end

# printing results
result = mostActiveRegion(500, -23.7222, -179.4605)
#println(result)
#=
result = mostActiveRegion(1000, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(3000, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(4000, -23.7222, -179.4605)
println(result)
=#

Row,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
,String31,Float64,Float64,Float64,Float64,String3,Int64?,Int64?,Float64?,Float64,String3,String15,String31,String,String15,Float64?,Float64,Float64?,Int64?,String15,String3,String3
1,2023-01-27T16:37:39,-23.7222,-179.46,525.778,4.6,mb,33,122,4.686,0.53,us,us6000jjsa,2023-01-27T17:00:08.040Z,south of the Fiji Islands,earthquake,16.53,18.134,0.1,30,reviewed,us,us
2,2023-01-27T16:33:28,-23.3301,-179.886,528.172,4.5,mb,28,115,4.841,0.64,us,us6000jjs9,2023-01-27T17:14:45.040Z,south of the Fiji Islands,earthquake,14.76,10.621,0.118,21,reviewed,us,us
3,2023-01-24T06:15:57,-22.1207,-175.436,43.479,4.7,mb,36,148,0.971,0.46,us,us6000jiv5,2023-01-24T06:49:05.040Z,"100 km SSW of ‘Ohonua, Tonga",earthquake,10.54,8.569,0.062,80,reviewed,us,us
4,2023-01-23T10:37:46,-23.3485,-179.55,565.089,4.6,mb,32,99,4.572,0.74,us,us7000j6by,2023-01-23T11:36:38.040Z,south of the Fiji Islands,earthquake,10.06,15.459,0.107,26,reviewed,us,us
5,2023-01-21T16:06:01,-27.3604,-176.596,10.0,4.8,mb,18,123,6.293,1.04,us,us6000jjqp,2023-01-27T21:01:52.040Z,Kermadec Islands region,earthquake,13.01,1.923,0.197,9,reviewed,us,us
6,2023-01-18T18:17:12,-24.1049,-179.526,505.884,4.5,mb,18,62,4.947,0.83,us,us6000jjf6,2023-01-27T21:36:56.040Z,south of the Fiji Islands,earthquake,7.34,10.414,0.157,12,reviewed,us,us
7,2023-01-13T05:43:10,-20.5228,-178.105,479.867,4.5,mb,32,104,2.788,0.61,us,us7000j5uf,2023-01-26T08:53:56.040Z,Fiji region,earthquake,13.77,5.531,0.111,24,reviewed,us,us
8,2023-01-12T14:16:22,-23.3615,-174.835,10.0,4.6,mb,20,95,2.208,0.81,us,us7000j5tt,2023-01-20T21:26:53.040Z,"224 km S of ‘Ohonua, Tonga",earthquake,8.56,1.847,0.158,12,reviewed,us,us
9,2023-01-12T12:28:33,-23.2748,-174.787,10.0,5.3,mww,45,87,2.13,1.41,us,us7000j3wd,2023-01-12T12:48:26.040Z,Tonga region,earthquake,8.23,1.882,0.093,11,reviewed,us,us


In [100]:
# Problem 10
function inTimeRange(M, x) #modified for date value instead of dataframe index for input
    present = Dates.now()
    # println((Dates.value(present) - Dates.value(DateTime(df.time[x], DateFormat("yyyy-mm-ddTHH:MM:SS")))))
    dayPassed = ((Dates.value(present) - Dates.value(DateTime(x, DateFormat("yyyy-mm-ddTHH:MM:SS.sssZ"))))  / (1000 * 3600 * 24))
    # println("the day passed for index $i is $dayPassed")
    if (dayPassed <= M)
        return true
    end
    return false
end

# modified for inTimeRange itegration
function highestMagnitude(df, M)
    highestMagnitude = 0
    result = ""
    for x in eachrow(df) 
        if (x.mag > highestMagnitude) && (inTimeRange(M, x.time))
            highestMagnitude = x.mag
            result = x.place
        end
    end
    return result
end

function mostActiveRegion(N, M, latitude , longitude)
    # searching for the most active region
    df_modified = filter(row -> isInRadius(row, N, latitude, longitude), df)
    df_mod3 = filter(row -> inTimeRange(M, row.time), df_modified)
    #finding the max maginitude region inside the radius and return its name
    df_mod4 = combine(groupby(df_mod3, :place), :mag => sum => :mag)
    mostActiveRegion = highestMagnitude(df_mod4)
    return mostActiveRegion
end



#printing results
result = mostActiveRegion(500, 10, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(500, 20, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(500, 30, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(1000, 10, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(1000, 20, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(1000, 30, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, 10, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, 20, -23.7222, -179.4605)
println(result)
result = mostActiveRegion(2000, 30, -23.7222, -179.4605)
println(result)



south of the Fiji Islands
south of the Fiji Islands

Kermadec Islands region
Kermadec Islands region



Kermadec Islands region
Kermadec Islands region


TU PHAM

MATT LE

MINH LE